In [2]:
import pandas as pd
import os
print(os.getcwd())
import pandas as pd
print(pd.__version__)
!python --version
!jupyter --version

# Load data from the input folder
enrollment_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Enrollment_Info_2015_01.csv")
service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")
contract_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Contract_Info_2015_01.csv", encoding='latin1')
pd.set_option('display.max_columns', None)



# Display the datasets
print(enrollment_data.head())
print(service_area_data.head())


/Users/ryanscholte/Desktop/GitHub/HW1/submission1
2.2.3
Python 3.10.16
Selected Jupyter core packages...
IPython          : 8.31.0
ipykernel        : 6.29.5
ipywidgets       : not installed
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : not installed
jupyterlab       : not installed
nbclient         : not installed
nbconvert        : not installed
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3
  Contract Number  Plan ID  SSA State County Code  FIPS State County Code  \
0           E0654      801                  58160                     NaN   
1           E0654      801                  61000                     NaN   
2           E0654      801                  96001                     NaN   
3           E0654      801                   2275                     NaN   
4           E0654      801                   2198                     NaN   

  State County Enrollment  
0   NaN    NaN     

/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_37931/4202740048.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")


In [ ]:
print(enrollment_data.columns)
print(contract_data.columns)
print(service_area_data.columns)


In [ ]:
#rename Contract Number to Contract ID , SSA State county code to SSA, and FIPS state county code to FIPS in enrollment data
enrollment_data.rename(columns={'Contract Number': 'Contract ID', 'SSA State County Code': 'SSA', 'FIPS State County Code': 'FIPS'}, inplace=True)

# only keep the columns we need "contractid", "planid", "ssa", "fips", "state", "county", "enrollment" for all sets
enrollment_data = enrollment_data[['Contract ID', 'Plan ID', 'SSA', 'FIPS', 'State', 'County', 'Enrollment']]
service_area_data = service_area_data[['Contract ID',  'Organization Name', 'Organization Type', 'Plan Type', 'SSA', 'FIPS', 'State', 'County']]



In [ ]:


# merge enrollment data with contract data on 'Contract ID' and 'Plan ID'
merged_data = pd.merge(enrollment_data, contract_data, on=['Contract ID', 'Plan ID'], how='left')

#show me the merged data head
print(merged_data.head())

In [ ]:
print(merged_data['Contract ID'].dtype)
print(service_area_data['Contract ID'].dtype)
merged_data['Contract ID'] = merged_data['Contract ID'].astype(str)
service_area_data['Contract ID'] = service_area_data['Contract ID'].astype(str)


In [ ]:
# merged_data = merged_data.apply(pd.to_numeric, downcast='integer', errors='ignore')
# service_area_data = service_area_data.apply(pd.to_numeric, downcast='integer', errors='ignore')

# chunk_size = 100000  # Adjust based on memory
# result = pd.DataFrame()

# for chunk in pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv", chunksize=chunk_size):
#     chunk_result = pd.merge(chunk, service_area_data, on='Contract ID', how='left')
#     result = pd.concat([result, chunk_result], ignore_index=True)

In [ ]:
#merge in service area data
# merged_data = pd.merge(merged_data, service_area_data, on='Contract ID', how='left')

In [ ]:

# # Group by Plan ID and count the occurrences for enrollment data
# enrollment_grouped = enrollment_data.groupby('Plan ID').size().reset_index(name='Count')

# # Group by Plan Type and count the occurrences for service area data
# service_area_grouped = service_area_data.groupby('Plan Type').size().reset_index(name='Count')

# # Set display options to view all columns
# pd.set_option('display.max_columns', None)

# # Display the grouped data
# print("Enrollment Data Grouped by Plan ID:")
# print(enrollment_grouped.head())

# print("\nService Area Data Grouped by Plan Type:")
# print(service_area_grouped.head())